In [6]:
import pandas as pd
import boto3
import io

In [7]:
# S3에 액세스할 수 있는 클라이언트 생성
s3 = boto3.client('s3')

# S3에서 파일 가져오기
response = s3.get_object(Bucket='dothis-ai-test', Key='data/channel_keyword_data_20230616.csv')

df = pd.read_csv(io.BytesIO(response["Body"].read()))

df

,channel_name,tf-idf-keywords
0,G마켓 옥션 판매자 교육센터 TV,"마켓,옥션,창업,오픈,광고,tv,판매,상품,ft,esm"
1,여의도 세력개미,"전망,주가,세력,긴급,목표,속보,매집,분석,공개,리튬"
2,정수TV,"학년,초등,학교,차시,설명,학기,영어,대교,수학,사회"
3,주식25,"주가,전망,주주,대박,매수,호재,상한가,축하,당장,필수"
4,나인트롯,"트롯,tv,조선,미스터,시청,홍지윤,방송,화요일,확정,프로그램"
...,...,...
6384,무쏘TV-김희천 대표,"가치주,금리,트윗,테마,중국,주식,종목,인상,육계,원전"
6385,심심한 회사원,"리얼돌,엄마"
6386,미정살롱,"교육,드라이,볼륨,사모님,인턴"
6387,내맘대로2,"shorts,결혼,권해,스킨,아내,윤지민"


In [13]:
data = list(set(",".join(df["tf-idf-keywords"].tolist()).split(",")))
len(data)

14141